# Ollama + OpenAI + Python

## 1. Especificar el nombre del modelo

Si has cargado un modelo diferente a "phi3:mini", cambia el valor en la celda de abajo. Esa variable se utilizará en el código a lo largo del cuaderno.


In [ ]:
MODEL_NAME = "phi3:mini"

## 2. Configurar el cliente de Open AI

Normalmente, el cliente de OpenAI se utiliza con OpenAI.com o Azure OpenAI para interactuar con modelos de lenguaje de gran tamaño.  
Sin embargo, también se puede usar con Ollama, ya que Ollama ofrece un endpoint compatible con OpenAI en "http://localhost:11434/v1".


In [ ]:
%pip install openai

In [ ]:
import openai

client = openai.OpenAI(
    base_url="http://localhost:11434/v1",
    api_key="nokeyneeded",
)

## 3. Generar una finalización de chat

Ahora podemos usar el SDK de OpenAI para generar una respuesta para una conversación. Esta solicitud debería generar un haiku sobre gatos:


In [ ]:
response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Write a haiku about a hungry cat"},
    ],
)

print("Response:")
print(response.choices[0].message.content)


## 4. Ingeniería de indicaciones

El primer mensaje enviado al modelo de lenguaje se llama "mensaje del sistema" o "indicación del sistema", y establece las instrucciones generales para el modelo.  
Puedes proporcionar tu propia indicación personalizada del sistema para guiar al modelo de lenguaje a generar respuestas de una manera diferente.  
Modifica el `SYSTEM_MESSAGE` a continuación para responder como tu personaje favorito de una película o serie de televisión, o inspírate en otras indicaciones del sistema de [Awesome ChatGPT Prompts](https://github.com/f/awesome-chatgpt-prompts?tab=readme-ov-file#prompts).

Una vez que hayas personalizado el mensaje del sistema, proporciona la primera pregunta del usuario en el `USER_MESSAGE`.


In [ ]:
SYSTEM_MESSAGE = """
I want you to act like Elmo from Sesame Street.
I want you to respond and answer like Elmo using the tone, manner and vocabulary that Elmo would use.
Do not write any explanations. Only answer like Elmo.
You must know all of the knowledge of Elmo, and nothing more.
"""

USER_MESSAGE = """
Hi Elmo, how are you doing today?
"""

response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": USER_MESSAGE},
    ],
)

print("Response:")
print(response.choices[0].message.content)


## 5. Ejemplos con pocos casos

Otra forma de guiar a un modelo de lenguaje es proporcionar "pocos casos", una secuencia de preguntas/respuestas de ejemplo que demuestren cómo debería responder.

El ejemplo a continuación intenta que un modelo de lenguaje actúe como un asistente de enseñanza proporcionando algunos ejemplos de preguntas y respuestas que un asistente podría dar, y luego se le plantea al modelo una pregunta que un estudiante podría hacer.

Pruébalo primero y luego modifica el `SYSTEM_MESSAGE`, `EXAMPLES` y `USER_MESSAGE` para un nuevo escenario.


In [ ]:
SYSTEM_MESSAGE = """
You are a helpful assistant that helps students with their homework.
Instead of providing the full answer, you respond with a hint or a clue.
"""

EXAMPLES = [
    (
        "What is the capital of France?",
        "Can you remember the name of the city that is known for the Eiffel Tower?"
    ),
    (
        "What is the square root of 144?",
        "What number multiplied by itself equals 144?"
    ),
    (   "What is the atomic number of oxygen?",
        "How many protons does an oxygen atom have?"
    ),
]

USER_MESSAGE = "What is the largest planet in our solar system?"


response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": EXAMPLES[0][0]},
        {"role": "assistant", "content": EXAMPLES[0][1]},
        {"role": "user", "content": EXAMPLES[1][0]},
        {"role": "assistant", "content": EXAMPLES[1][1]},
        {"role": "user", "content": EXAMPLES[2][0]},
        {"role": "assistant", "content": EXAMPLES[2][1]},
        {"role": "user", "content": USER_MESSAGE},
    ],
)


print("Response:")
print(response.choices[0].message.content)

## 6. Generación Aumentada por Recuperación

RAG (Generación Aumentada por Recuperación) es una técnica para lograr que un modelo de lenguaje responda preguntas de manera precisa en un dominio específico, primero recuperando información relevante de una fuente de conocimiento y luego generando una respuesta basada en esa información.

Hemos proporcionado un archivo CSV local con datos sobre autos híbridos. El código a continuación lee el archivo CSV, busca coincidencias con la pregunta del usuario y luego genera una respuesta basada en la información encontrada. Ten en cuenta que esto tomará más tiempo que cualquiera de los ejemplos anteriores, ya que envía más datos al modelo. Si notas que la respuesta aún no está basada en los datos, puedes intentar ingeniería de sistemas o probar con otros modelos. En general, RAG es más efectivo con modelos más grandes o con versiones ajustadas de SLMs.


In [ ]:
import csv

SYSTEM_MESSAGE = """
You are a helpful assistant that answers questions about cars based off a hybrid car data set.
You must use the data set to answer the questions, you should not provide any information that is not in the provided sources.
"""

USER_MESSAGE = "how fast is a prius?"

# Open the CSV and store in a list
with open("hybrid.csv", "r") as file:
    reader = csv.reader(file)
    rows = list(reader)

# Normalize the user question to replace punctuation and make lowercase
normalized_message = USER_MESSAGE.lower().replace("?", "").replace("(", " ").replace(")", " ")

# Search the CSV for user question using very naive search
words = normalized_message.split()
matches = []
for row in rows[1:]:
    # if the word matches any word in row, add the row to the matches
    if any(word in row[0].lower().split() for word in words) or any(word in row[5].lower().split() for word in words):
        matches.append(row)

# Format as a markdown table, since language models understand markdown
matches_table = " | ".join(rows[0]) + "\n" + " | ".join(" --- " for _ in range(len(rows[0]))) + "\n"
matches_table += "\n".join(" | ".join(row) for row in matches)
print(f"Found {len(matches)} matches:")
print(matches_table)

# Now we can use the matches to generate a response
response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": USER_MESSAGE + "\nSources: " + matches_table},
    ],
)

print("Response:")
print(response.choices[0].message.content)


---

**Descargo de responsabilidad**:  
Este documento ha sido traducido utilizando el servicio de traducción automática [Co-op Translator](https://github.com/Azure/co-op-translator). Si bien nos esforzamos por garantizar la precisión, tenga en cuenta que las traducciones automatizadas pueden contener errores o imprecisiones. El documento original en su idioma nativo debe considerarse la fuente autorizada. Para información crítica, se recomienda una traducción profesional realizada por humanos. No nos hacemos responsables de malentendidos o interpretaciones erróneas que puedan surgir del uso de esta traducción.
